# Сравнение алгоритмов кластеризации по метрикам качества кластеризации

In [1]:
import json
import csv
import time

import numpy as np

from itertools import groupby
from collections import Counter

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
from sklearn.cluster import AffinityPropagation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

from sklearn import metrics

## Подготовка данных для кластеризации

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

## Загрузка тестовой выборки

In [3]:
news = []
with open('/data/10k.test.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []
for n in news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

news_content = [x.content for x in news]

total words: 2717122


In [5]:
stopwords = set(one_time)

## Загрузка размеченной выборки

In [6]:
marked_map = {} # (id, label)
with open('/data/mark_news.csv', encoding="utf8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        marked_map[row[0]] = int(row[3])

marked_news = []
for n in news:
    label = marked_map[n.id]
    marked_news.append((n.id, label))
    
marked_labels = [label for n_id, label in marked_news]

In [7]:
marked_map['170bf9b9-d62d-437e-a75a-cac7b7c9f282']

40

# Вспомогательные функции

In [8]:
def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def filter_words(text):
    words_list = text.split()
    newWords = [x for x in words_list if len(x) > 3]
    return " ".join(newWords)

def print_clusters(cluster_news, clustre_labels):
    newsLabels = zip_news(cluster_news, clustre_labels)
    newsLabels = sorted(newsLabels, key=lambda n: n.label)
    for label, group in groupby(newsLabels, lambda n: n.label):
        groupList = list(group)
        print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
        for gr in groupList:
            print("\t" + gr.title)
            
def print_topics(components, feature_names, n_top_words):
    for topic_idx, topic in enumerate(components):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Векторизация

## TF-IDF

In [9]:
def tf_idf(content):
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)
    tfidf_matrix = tfidf_vectorizer.fit_transform(content)
    print("vocabulary size: %s" % len(tfidf_vectorizer.vocabulary_))
    return tfidf_matrix

## TF

In [10]:
def tf(content):
    tf_vectorizer = TfidfVectorizer(use_idf=False, tokenizer=lambda text: text.split(" "), stop_words=stopwords) # , ngram_range=(1, 3)
    tf_matrix = tfidf_vectorizer.fit_transform(content)
    print("vocabulary size: %s" % len(tf_vectorizer.vocabulary_))
    return tf_matrix

# Семантический анализ

## LDA

In [11]:
def lda(content, n, max_iter):
    tf = CountVectorizer(stop_words=stopwords).fit_transform(content)
    lda = LatentDirichletAllocation(n_topics=n, max_iter=max_iter, learning_method='online', learning_offset=50.)
    lda_matrix = lda.fit_transform(tf)
    normalizer = Normalizer()
    norm_matrix = normalizer.fit_transform(lda_matrix)
    return norm_matrix

## LSA

In [12]:
def lsa(matrix, n, max_inter):
    lsa = TruncatedSVD(n_components=n, n_iter=max_inter)
    lsa_matrix = lsa.fit_transform(matrix)
    return lsa_matrix

# Кластеризация

# Проверка качества кластеризации

In [13]:
def score(matrix, marked_labels, clustered_labels):
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(marked_labels, clustered_labels))
    print("Completeness: %0.3f" % metrics.completeness_score(marked_labels, clustered_labels))
    print("V-measure: %0.3f" % metrics.v_measure_score(marked_labels, clustered_labels))
    print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(marked_labels, clustered_labels))
    print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(matrix, clustered_labels, sample_size=1000))
    print("Cluster count: %s" % len(set(clustered_labels)))

# Тест 

In [14]:
max_iter = 100 # 100
n_components = 10000
n_clusters = 130 # Количество кластеров
min_samples = 4
eps = 1.3

In [16]:
tfidf_matrix = tf_idf(news_content)

vocabulary size: 15558


In [18]:
vectorization = {
    "tf-idf": tfidf_matrix,
    "tf": tf_idf(news_content),
    "lda": lda(news_content, n_components, max_iter),
    "lsa": lsa(tfidf_matrix, n_components, max_iter)
}

vocabulary size: 15558


In [19]:
clasterization = {
    "kmeans": KMeans(n_clusters=n_clusters),
    "miniBatchKMeanss": MiniBatchKMeans(n_clusters=n_clusters, max_iter=max_iter),
    "dbscan": DBSCAN(eps=eps, min_samples=min_samples),
    "birch": Birch(n_clusters=n_clusters),
    "affinityPropagation": AffinityPropagation()
}

In [20]:
for cl_name, cl_alg in clasterization.items():
    for vec_name, vec_matrix in vectorization.items():
        try:
            print("%s %s" % (cl_name, vec_name))
            start_time = time.time()
            result_matrix = cl_alg.fit(vec_matrix)
            labels = result_matrix.labels_
            score(vec_matrix, marked_labels, labels)
            print("%s %s: %s second" % (cl_name, vec_name, time.time() - start_time))
            print("----")
        except Exception as ex:
            print(ex)

kmeans tf-idf
Homogeneity: 0.859
Completeness: 0.690
V-measure: 0.766
Adjusted Rand-Index: 0.251
Silhouette Coefficient: 0.070
Cluster count: 130
kmeans tf-idf: 8.435837030410767 second
----
kmeans tf
Homogeneity: 0.853
Completeness: 0.685
V-measure: 0.760
Adjusted Rand-Index: 0.246
Silhouette Coefficient: 0.064
Cluster count: 130
kmeans tf: 8.243932008743286 second
----
kmeans lda
Homogeneity: 0.589
Completeness: 0.486
V-measure: 0.532
Adjusted Rand-Index: 0.003
Silhouette Coefficient: 0.535
Cluster count: 130
kmeans lda: 0.7911496162414551 second
----
kmeans lsa
Homogeneity: 0.853
Completeness: 0.682
V-measure: 0.757
Adjusted Rand-Index: 0.216
Silhouette Coefficient: 0.285
Cluster count: 130
kmeans lsa: 0.8746170997619629 second
----
miniBatchKMeanss tf-idf
Homogeneity: 0.577
Completeness: 0.648
V-measure: 0.610
Adjusted Rand-Index: 0.053
Silhouette Coefficient: -0.016
Cluster count: 125
miniBatchKMeanss tf-idf: 1.9130744934082031 second
----
miniBatchKMeanss tf
Homogeneity: 0.567
Co

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cluster/birch.py:602: UserWarning: Number of subclusters found (1) by Birch is less than (130). Decrease the threshold.
  % (len(centroids), self.n_clusters))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cluster/birch.py:602: UserWarning: Number of subclusters found (96) by Birch is less than (130). Decrease the threshold.
  % (len(centroids), self.n_clusters))


Silhouette Coefficient: 0.214
Cluster count: 96
birch lsa: 0.2521991729736328 second
----
affinityPropagation tf-idf
Homogeneity: 0.816
Completeness: 0.713
V-measure: 0.761
Adjusted Rand-Index: 0.350
Silhouette Coefficient: 0.064
Cluster count: 86
affinityPropagation tf-idf: 0.8325362205505371 second
----
affinityPropagation tf
Homogeneity: 0.816
Completeness: 0.713
V-measure: 0.761
Adjusted Rand-Index: 0.350
Silhouette Coefficient: 0.064
Cluster count: 86
affinityPropagation tf: 0.8357975482940674 second
----
affinityPropagation lda
Homogeneity: 0.399
Completeness: 0.454
V-measure: 0.424
Adjusted Rand-Index: 0.013


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:204: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


Silhouette Coefficient: 0.312
Cluster count: 128
affinityPropagation lda: 0.9901378154754639 second
----
affinityPropagation lsa
Homogeneity: 0.754
Completeness: 0.693
V-measure: 0.722
Adjusted Rand-Index: 0.228
Silhouette Coefficient: 0.265
Cluster count: 81
affinityPropagation lsa: 0.40547633171081543 second
----


## LDA

In [21]:
start_time = time.time()
lda_matrix = lda(news_content, n_clusters, max_iter)
lda_labels = [x.argmax() for x in lda_matrix]
score(lda_matrix, marked_labels, lda_labels)
print("lda lda: %s second" % (time.time() - start_time))

Homogeneity: 0.170
Completeness: 0.773
V-measure: 0.279
Adjusted Rand-Index: 0.054
Silhouette Coefficient: 0.803
Cluster count: 9
lda lda: 12.823763608932495 second


## LSA

In [22]:
start_time = time.time()
lsa_matrix = lsa(tfidf_matrix, n_clusters, max_iter)
lsa_labels = [x.argmax() for x in lsa_matrix]
score(lsa_matrix, marked_labels, lsa_labels)
print("lsa: %s second" % (time.time() - start_time))

Homogeneity: 0.688
Completeness: 0.714
V-measure: 0.701
Adjusted Rand-Index: 0.225
Silhouette Coefficient: 0.045
Cluster count: 109
lsa: 1.0879974365234375 second
